In [1]:
import pandas as pd 

path = 'reg_data'
dataset = ['reg1.csv', 'reg2.csv', 'reg3.csv']
df = pd.DataFrame()
for i in dataset:
    tmp = pd.read_csv(f'{path}/{i}')
    df = pd.concat([df, tmp])
    df = df[['batch_size', 'learning_rate','dropout', 'loss', 'val_loss']].dropna()
df['mean'] = (df['loss'] + df['val_loss'])/2
df

,batch_size,learning_rate,dropout,loss,val_loss,mean
0,182,0.005448,0.694351,0.078256,0.041367,0.059811
1,119,0.003378,0.493624,0.059851,0.039452,0.049652
2,115,0.004924,0.684175,0.062070,0.040731,0.051400
3,181,0.003706,0.536575,0.068155,0.040875,0.054515
4,143,0.004607,0.573039,0.062509,0.041824,0.052166
...,...,...,...,...,...,...
14,41,0.007689,0.038130,0.081920,0.065419,0.073669
15,25,0.006920,0.144256,0.051570,0.060358,0.055964
16,58,0.007874,0.317321,0.081749,0.059558,0.070653
17,35,0.001483,0.302686,0.060643,0.059847,0.060245


In [2]:
min_df = df[df['mean'] < 0.041]
min_df

,batch_size,learning_rate,dropout,loss,val_loss,mean
24,77,0.000317,0.057405,0.040433,0.040049,0.040241


In [3]:
import plotly.express as px 
import plotly.graph_objects as go 

fig = go.Figure()
fig.add_traces(
    go.Mesh3d(
        x=df['learning_rate'], y=df['dropout'], z=df['mean'], 
        opacity=1,
        intensity=df['batch_size'], colorbar_title='batch_size',
        colorscale='viridis', 
        )
    )
fig.add_traces(
    go.Scatter3d(
        x=min_df['learning_rate'], y=min_df['dropout'], z=min_df['mean'],
        marker=dict(size=6, color='red')
    )
)

fig.update_layout(
    scene=dict(
        xaxis_title='learning_rate',
        yaxis_title='dropout',
        zaxis_title='val_loss',
        # zaxis=dict(range=[0.02, 0.2])
        # xaxis=dict(range=[0, 0.01])
    )
)
fig.show()

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np 


x = df[['batch_size', 'learning_rate', 'dropout']]
y = df['val_loss']
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1)



In [5]:
degree = 2
poly_reg = PolynomialFeatures(degree=degree)
poly_x_train = poly_reg.fit_transform(x_train)
regr = LinearRegression()
regr.fit(poly_x_train, y_train)
print(x_train.iloc[0])
print(poly_x_train[0])


batch_size       28.000000
learning_rate     0.000197
dropout           0.156837
Name: 36, dtype: float64
[1.00000000e+00 2.80000000e+01 1.96977005e-04 1.56836806e-01
 7.84000000e+02 5.51535613e-03 4.39143058e+00 3.87999403e-08
 3.08932443e-05 2.45977838e-02]


In [6]:
coef = regr.coef_
'''
coef of model, degree = 2: [1 (1),  a (2),  b (3),  c (4),
                            a^2 (5), ab (6), ac (7), b^2 (8), 
                            bc (9),  c^2 (10)]
'''
coef

array([ 0.00000000e+00, -1.25567569e-04,  2.94656775e+00, -2.16284477e-02,
        4.04551910e-07,  7.69695233e-03, -8.47959681e-05, -6.93214439e+01,
       -5.90850691e+00,  4.35162533e-02])

In [9]:
# poly_x_val = poly_reg.fit_transform(x_val)
# poly_y_val = regr.predict(poly_x_val)
# mse = mean_squared_error(poly_y_val, y_val)
# print(mse)
# def prep(arr):
#     arr[-1] *= 1/12
#     return arr

In [10]:
'''
beta 1->5: same
beta6 = coef[7]
beta7 = coef[9]
beta8 = coef[5]
beta9 = coef[6]
beta10 = coef[8]
'''
coef_matrix = np.array([[2*coef[4], coef[5], coef[6]],
                        [coef[5], 2*coef[7], coef[8]],
                        [coef[6], coef[8], 2*coef[9]]])
depen_var = np.array([-coef[1], -coef[2], -coef[3]])
res = np.linalg.solve(coef_matrix, depen_var)
res = prep(res)
res

array([1.92684079e+02, 3.43133432e-03, 5.57658910e-02])

In [11]:
# test = 2*coef[4]*res[0] + coef[5]*res[1] + coef[6]*res[-1]
# print(test, ' ', -coef[1])


In [12]:
poly_res = poly_reg.fit_transform(res.reshape(1,-1))
regr.predict(poly_res)

array([0.05620648])